In [1]:
import pandas as pd
import json

In [21]:
index = ['measles','GP','hosp','death']
data = {'0-4': [28730,11883,7470,97],
        '5-9': [6492, 1173, 612, 9],
        '10-19': [18580, 2369, 1612, 18],
        '20-29': [9161, 2656, 2075, 26],
        '30-': [4069, 1399, 1107, 27]}
columns = list(data.keys())

df = pd.DataFrame(data, index=index)

row_length = len(index) + 1

df

,0-4,5-9,10-19,20-29,30-
measles,28730,6492,18580,9161,4069
GP,11883,1173,2369,2656,1399
hosp,7470,612,1612,2075,1107
death,97,9,18,26,27


In [22]:
print((28730 - 11883)/28730)
print((2369 - 1612) / 18580)
print(26 / 9161)

0.5863905325443787
0.04074273412271259
0.0028381181093767058


In [23]:
def process_rows(row1, row2, measles, final_row=False):

    if final_row:

        return row1 / measles

    else:

        return (row1 - row2) / measles

In [24]:
prob_data = []
prob_index = ['prop_final_mild', 'prop_final_gp','prop_final_hosp','prop_final_death']


for i in range(len(df)):
    
    if i == len(df) - 1:

        new_row = process_rows(df.iloc[i], None, df.iloc[0], final_row=True)

    else:

        new_row = process_rows(df.iloc[i], df.iloc[i+1], df.iloc[0], final_row=False)

    prob_data.append(new_row)

prob_df = pd.DataFrame(prob_data, index=prob_index, columns=df.columns)
prob_df

,0-4,5-9,10-19,20-29,30-
prop_final_mild,0.586391,0.819316,0.872497,0.710075,0.656181
prop_final_gp,0.153603,0.086414,0.040743,0.063421,0.071762
prop_final_hosp,0.256631,0.092884,0.085791,0.223666,0.265421
prop_final_death,0.003376,0.001386,0.000969,0.002838,0.006636


In [25]:
prob_hosp_given_measles = [df.loc['hosp', columns[i]] / df.loc['measles', columns[i]] for i in range(row_length)]
print(prob_hosp_given_measles)
prob_gp_given_measles = [df.loc['GP', columns[i]] / df.loc['measles', columns[i]] for i in range(row_length)]
prob_death_given_measles = [df.loc['death', columns[i]] / df.loc['measles', columns[i]] for i in range(row_length)]

prob_gp_to_hosp = [prob_hosp_given_measles[i] / prob_gp_given_measles[i] for i in range(row_length)]
prob_gp_to_recov = [1 - prob_gp_to_hosp[i] for i in range(row_length)]

prob_hosp_to_death = [prob_death_given_measles[i] / prob_hosp_given_measles[i] for i in range(row_length)]

prob_hosp_to_icu = [0.1, 0.1, 0.1, 0.1, 0.1] #changed

prob_exposed_to_asympt = [0.1, 0.1, 0.1, 0.1, 0.9] #looked at further

prob_icu_to_death = [4/15 for _ in range(row_length)] #changed

prob_icu_to_recov = [1 - prob_icu_to_death[i] for i in range(row_length)]

prob_hosp_to_recov = [1 - prob_hosp_to_death[i] - prob_hosp_to_icu[i] for i in range(row_length)]

prob_exposed_to_mild =  prob_df.loc['prop_final_mild'].tolist()
prob_exposed_to_mild = [(1 - prob_exposed_to_asympt[i]) * prob_exposed_to_mild[i] for i in range(row_length)]

prob_exposed_to_gp =  prob_df.loc['prop_final_mild'].tolist()
prob_exposed_to_gp = [(1 - prob_exposed_to_asympt[i]) * (1 - prob_exposed_to_gp[i]) for i in range(row_length)]


[0.2600069613644274, 0.09426987060998152, 0.08675995694294941, 0.22650365680602555, 0.27205701646596214]


In [26]:
# Sample dictionary
save_data = {
    'prob_gp_to_hosp': prob_gp_to_hosp,
    'prob_gp_to_recov': prob_gp_to_recov,
    'prob_exposed_to_asympt': prob_exposed_to_asympt,
    'prob_exposed_to_gp': prob_exposed_to_gp,
    'prob_exposed_to_mild': prob_exposed_to_mild,
    'prob_hosp_to_death': prob_hosp_to_death,
    'prob_hosp_to_recov': prob_hosp_to_recov,
    'prob_hosp_to_icu': prob_hosp_to_icu,
    'prob_icu_to_death': prob_icu_to_death,
    'prob_icu_to_recov': prob_icu_to_recov
}

with open("probabilities.json", 'w') as json_file:

    json.dump(save_data, json_file)

print("Data saved as probabilities.json")

Data saved as probabilities.json


In [34]:
index = list(save_data.keys())
probabilities_df = pd.DataFrame(save_data, index=columns, columns=index)
probabilities_df = probabilities_df.T
print(probabilities_df)



                             0-4       5-9     10-19     20-29       30-
prob_gp_to_hosp         0.628629  0.521739  0.680456  0.781250  0.791279
prob_gp_to_recov        0.371371  0.478261  0.319544  0.218750  0.208721
prob_exposed_to_asympt  0.100000  0.100000  0.100000  0.100000  0.900000
prob_exposed_to_gp      0.372249  0.162616  0.114752  0.260932  0.034382
prob_exposed_to_mild    0.527751  0.737384  0.785248  0.639068  0.065618
prob_hosp_to_death      0.012985  0.014706  0.011166  0.012530  0.024390
prob_hosp_to_recov      0.887015  0.885294  0.888834  0.887470  0.875610
prob_hosp_to_icu        0.100000  0.100000  0.100000  0.100000  0.100000
prob_icu_to_death       0.266667  0.266667  0.266667  0.266667  0.266667
prob_icu_to_recov       0.733333  0.733333  0.733333  0.733333  0.733333
